# 03 Algorithms

## 3.15 Policy Gradient Methods

### Metrics of Policy

在此之前，我们已经介绍了使用神经网络近似状态价值或动作价值的算法，如DQN。然而，在许多实际应用中，我们更关心的是如何学习到一个策略（policy），即从状态到动作的映射。这种情况下，我们可以使用策略梯度方法（Policy Gradient Methods）来直接对策略进行优化。

假设$J(\theta)$是一个标量，用于表示策略$\pi$的性能度量，我们的目标是最大化这个目标函数。根据基于梯度的算法，我们可以通过以下公式更新参数$\theta$：
$$
\theta_{t+1} = \theta_t + \alpha \nabla J(\theta)
$$

一般地，策略的性能度量基于状态价值期望或奖励期望。这里我们对于这两种性能度量分别做以下定义：
| Metric | Expression 1                | Expression 2             | Expression 3                                                                 |
|--------|-----------------------------|--------------------------|------------------------------------------------------------------------------|
| $ \bar{v}_\pi $ | $ \sum_{s \in S} d(s) v_\pi(s) $ | $ E_{S \sim d}[v_\pi(S)] $  | $ \lim_{n \to \infty} E \left[ \sum_{t=0}^{n} \gamma^t R_{t+1} \right] $  |
| $ \bar{r}_\pi $ | $ \sum_{s \in S} d_\pi(s) r_\pi(s) $ | $ E_{S \sim d_\pi}[r_\pi(S)] $ | $ \lim_{n \to \infty} \frac{1}{n} E \left[ \sum_{t=0}^{n-1} R_{t+1} \right] $  |

其中，$d(s)$ 是是状态$s$的权重，满足$d(s) \geq 0$和$\sum_{a \in \cal S}$=1，一般地可以定义为均匀分布（$d_0(s)=\frac{1}{\cal |S|}$）或平稳分布($d_{\pi}^T P_{\pi} = d_{\pi}^T$，$P_{\pi}$是状态转移矩阵)。

上述所有指标都是策略$\pi$的函数，而策略$\pi$又被参数化表示为$\theta$，因此我们可以将这些指标表示为$\theta$的函数。简言之，不同的参数$\theta$对应着不同的指标值.

另外，状态价值期望$\bar v_{\pi}$和奖励期望$\bar r_{\pi}$之间的关系可以表示如下：
$$ \bar r_{\pi} = (1 - \gamma) \bar v_{\pi} $$













### Gradients of the Metrics

令
$$
J(\theta) = \sum_{s \in \cal S} \eta (s) v_{\pi}(s) = \sum_{s \in \cal S} \eta (s) \sum_{a \in \cal A} \pi(a|s;\theta) q_{\pi}(s, a)
$$
其中$\eta(s)$是平稳分布，$v_{\pi}(s)$是状态价值函数、$q_{\pi}(s, a)$是动作价值函数、$\pi(a|s;\theta)$是策略函数。

对于这个目标函数$J(\theta)$，我们可以计算它的梯度：
$$
\nabla_{\theta} J(\theta) = \sum_{s \in \cal S} \eta (s) \left( \sum_{a \in \cal A} q_{\pi}(s, a) \nabla_{\theta} \pi(a|s;\theta) \right)  \ \ \ \ \  *
$$

进一步地，由
$$
\nabla_{\theta} \log \pi(a|s;\theta) = \frac{\nabla_{\theta} \pi(a|s;\theta)}{\pi(a|s;\theta)}
$$
知：
$$
\nabla_{\theta} \pi(a|s;\theta) = \nabla_{\theta} \log \pi(a|s;\theta) \pi(a|s;\theta)
$$

代入(*)式中，得：
$$
\begin{aligned}
\nabla_{\theta} J(\theta) &= \sum_{s \in \cal S} \eta (s) \left( \sum_{a \in \cal A} q_{\pi}(s, a) \nabla_{\theta} \log \pi(a|s;\theta) \pi(a|s;\theta) \right) \\
&= \mathbb{E}_{S \sim \eta, A \sim \pi(S, \theta)} \Big[q_{\pi}(S, A) \nabla_{\theta} \log \pi(A|S;\theta) \Big] \\

\end{aligned}
$$


